In [2]:
import pandas as pd

In [3]:
from pathlib import Path

In [4]:
TRAIN_PATH = r"C:\Users\David\OneDrive\Desktop\4th year subjects\Machine Learning\wildfires_training.csv"
TEST_PATH  = r"C:\Users\David\OneDrive\Desktop\4th year subjects\Machine Learning\wildfires_test.csv"

In [5]:
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

In [6]:
from sklearn.preprocessing import OrdinalEncoder

In [7]:
on_fire = ['no', 'yes']

In [8]:
enc = OrdinalEncoder(categories=[on_fire])

In [9]:
train_df[['fire']] = enc.fit_transform(train_df[['fire']])

In [10]:
test_df[['fire']] = enc.transform(test_df[['fire']])

In [11]:
train_df.head(), test_df.head()

(   fire  year  temp  humidity  rainfall  drought_code  buildup_index  day  \
 0   0.0  2015    28        59       0.0          8.06           3.47    1   
 1   0.0  2010    30        61       1.3          8.17           4.03    2   
 2   0.0  2009    26        83      13.1          8.08           3.59    3   
 3   0.0  2017    25        87       2.5          7.18           2.42    4   
 4   0.0  2014    28        77       0.0         14.98           4.63    5   
 
    month  wind_speed  
 0      6          19  
 1      6          13  
 2      6          22  
 3      6          15  
 4      6          18  ,
    fire  year  temp  humidity  rainfall  drought_code  buildup_index  day  \
 0   0.0  2015    33        68       4.5          9.12           5.09   19   
 1   1.0  2009    28        56       0.0         38.17          21.21   12   
 2   0.0  2017    30        64       0.6         15.38           6.24   24   
 3   0.0  2007    23        74       8.3          7.36           2.27   1